In [2]:
# Full end-to-end implementation:
# Urdu Conversational Chatbot: Transformer w/ Multi-Head Attention
# Option A params: 2 encoder/2 decoder layers, d_model=256, heads=2
# Environment: Colab or Kaggle. Run on GPU for full training.

# ----------------------------
# 0. Install dependencies
# ----------------------------
# Run this cell in Colab (uncomment install lines if needed)
!pip install -q transformers sentencepiece sacrebleu rouge_score nltk streamlit pyngrok tqdm

import os, sys, math, time, random, json, re
from collections import Counter
from typing import List, Tuple, Optional
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

# Evaluation libs
import sacrebleu
from rouge_score import rouge_scorer
import nltk
nltk.download('punkt', quiet=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 119.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 112.9 MB/s eta 0:00:00


True

In [17]:
# ----------------------------
# 1. Configuration (Adjusted based on user guidelines)
# ----------------------------
config = {
    "KAGGLE_DATASET": "muhammadahmedansari/urdu-dataset-20000",
    "DATA_CSV": "urdu-dataset/urdu_dialog_dataset.csv",  # path inside dataset (will check)
    "MODEL_SAVE_PATH": "best_transformer_urdu.pt",
    "VOCAB_PATH": "vocab_tokens.json",
    "BATCH_SIZE": 32, # Within suggested range (32/64)
    "NUM_EPOCHS": 20, # Keep epochs same for now
    "LEARNING_RATE": 3e-4, # Within suggested range (1e-4 - 5e-4)
    # Adjusted parameters based on user guidelines
    "D_MODEL": 512, # Within suggested range (256/512)
    "NUM_HEADS": 2, # Fixed as per user guidelines
    "NUM_ENCODER_LAYERS": 2, # Fixed as per user guidelines
    "NUM_DECODER_LAYERS": 2, # Fixed as per user guidelines
    "D_FF": 2048, # Typically 4*D_MODEL
    "DROPOUT": 0.1, # Within suggested range (0.1 - 0.3)
    "MAX_LEN": 50,
    "MIN_FREQ": 2,
    "DEVICE": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "PAD_TOKEN":"[PAD]",
    "CLS_TOKEN":"[CLS]",
    "SEP_TOKEN":"[SEP]",
    "EOS_TOKEN":"[EOS]",
    "UNK_TOKEN":"[UNK]",
    "SAVE_BEST_BY":"bleu"  # validation metric for saving best model
}

print("Device:", config["DEVICE"])

Device: cuda


In [18]:
# ----------------------------
# 2. Download dataset (Kaggle)
# ----------------------------
import os
import pandas as pd
import kagglehub
import warnings

warnings.simplefilter(action='ignore', category=RuntimeWarning)

# ------------------------------------------
# 2. Download or Load Urdu Dataset (Auto)
# ------------------------------------------
def load_urdu_dataset():
    try:
        print("📦 Downloading Urdu conversational dataset using kagglehub...")
        # This automatically downloads to a cached local directory
        path = kagglehub.dataset_download("muhammadahmedansari/urdu-dataset-20000")
        print("✅ Dataset downloaded successfully.")
        print("📁 Path to dataset files:", path)

        # locate dataset file (adjust if filename differs)
        dataset_file = os.path.join(path, "final_main_dataset.tsv")
        if not os.path.exists(dataset_file):
            raise FileNotFoundError("final_main_dataset.tsv not found in the downloaded dataset.")

        # load using pandas
        d = pd.read_csv(dataset_file, sep="\t", encoding="utf-8")
        print(f"✅ Dataset loaded with {len(d)} rows and {len(d.columns)} columns.")
        return d

    except Exception as e:
        print("⚠️ Could not automatically load dataset:", e)
        print("👉 Please upload manually if running offline:")
        print("""
        from google.colab import files
        uploaded = files.upload()
        os.makedirs('data', exist_ok=True)
        !mv final_main_dataset.tsv data/
        d = pd.read_csv('data/final_main_dataset.tsv', sep='\\t', encoding='utf-8')
        d.head()
        """)
        return None

# Run the dataset loader
d = load_urdu_dataset()



📦 Downloading Urdu conversational dataset using kagglehub...
Using Colab cache for faster access to the 'urdu-dataset-20000' dataset.
✅ Dataset downloaded successfully.
📁 Path to dataset files: /kaggle/input/urdu-dataset-20000
✅ Dataset loaded with 20000 rows and 11 columns.


In [53]:
print(d.columns)

Index(['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age',
       'gender', 'accents', 'variant', 'locale', 'segment'],
      dtype='object')


In [19]:
import os
import pandas as pd

# Automatically locate dataset path if downloaded via kagglehub
# Use the path returned by kagglehub.dataset_download in the previous cell
path = '/kaggle/input/urdu-dataset-20000'
dataset_dir = path # 'path' is the variable from the previous cell

print(f"🔍 Searching inside: {dataset_dir}\n")

# List all files for transparency
for root, dirs, files in os.walk(dataset_dir):
    for f in files:
        print("📄", os.path.join(root, f))

# ----------------------------
# Locate and load dataset file with 'sentence' column
# ----------------------------
def find_sentence_dataset_file(base_dir='.'):
    """
    Search recursively for Urdu dataset (CSV or TSV)
    with a 'sentence' column.
    """
    for root, dirs, files in os.walk(base_dir):
        for f in files:
            if f.endswith(('.csv', '.tsv')):
                p = os.path.join(root, f)
                try:
                    sep = '\t' if f.endswith('.tsv') else ','
                    # Read a small sample to check columns
                    df_sample = pd.read_csv(p, nrows=5, sep=sep, encoding='utf-8')
                    cols = set(df_sample.columns.str.lower())
                    if 'sentence' in cols:
                        return p
                except Exception as e:
                    print(f"⚠️ Skipping {f} due to read error: {e}")
                    continue
    return None

csv_path = find_sentence_dataset_file(dataset_dir)
if csv_path is None:
    raise FileNotFoundError(
        f"❌ Could not find any .csv/.tsv file with a 'sentence' column under {dataset_dir}.\n"
        f"Please check the listed files above."
    )

print(f"\n✅ Found dataset file with 'sentence' column: {csv_path}")

# Load the full dataset
sep = '\t' if csv_path.endswith('.tsv') else ','
df = pd.read_csv(csv_path, sep=sep, encoding='utf-8')

# Keep only the 'sentence' column and drop missing values
df = df[['sentence']].dropna().reset_index(drop=True)
df.columns = ['sentence']

# Create conversational pairs: sentence -> sentence
# This approach is for training a generative model on single-turn data
# If you have actual question-answer pairs, the logic here would need adjustment.
df['question'] = df['sentence']
df['answer'] = df['sentence']
df = df[['question', 'answer']]


print(f"\n✅ Loaded rows: {len(df)}")
print(df.head(5))

Streaming output truncated to the last 5000 lines.
📄 /kaggle/input/urdu-dataset-20000/limited_wav_files/limited_wav_files/common_voice_ur_31348861.wav
📄 /kaggle/input/urdu-dataset-20000/limited_wav_files/limited_wav_files/common_voice_ur_31934786.wav
📄 /kaggle/input/urdu-dataset-20000/limited_wav_files/limited_wav_files/common_voice_ur_31922207.wav
📄 /kaggle/input/urdu-dataset-20000/limited_wav_files/limited_wav_files/common_voice_ur_31948829.wav
📄 /kaggle/input/urdu-dataset-20000/limited_wav_files/limited_wav_files/common_voice_ur_31823444.wav
📄 /kaggle/input/urdu-dataset-20000/limited_wav_files/limited_wav_files/common_voice_ur_31962481.wav
📄 /kaggle/input/urdu-dataset-20000/limited_wav_files/limited_wav_files/common_voice_ur_32001782.wav
📄 /kaggle/input/urdu-dataset-20000/limited_wav_files/limited_wav_files/common_voice_ur_31918464.wav
📄 /kaggle/input/urdu-dataset-20000/limited_wav_files/limited_wav_files/common_voice_ur_31932794.wav
📄 /kaggle/input/urdu-dataset-20000/limited_wav_fi

In [20]:
# ----------------------------
# 4. Urdu Normalizer (strict)
# ----------------------------
class UrduNormalizer:
    def __init__(self):
        # harakat/diacritics block
        self.diacritics_pattern = re.compile(r"[\u064B-\u0652\u0670\u0640]")
        # normalize different alef forms to 'ا'
        self.alef_pattern = re.compile(r"[\u0622\u0623\u0625\u0671\u0672\u0673]")
        # normalize Yeh forms to 'ی' (and fallback arabic yeh)
        self.yeh_pattern = re.compile(r"[\u0626\u06CC\u06C0\u06C1\u06C2\u064A]")
        # normalize different KAF forms to 'ک' (optional)
        self.kaf_pattern = re.compile(r"[\u0643\u06AA]")
        # punctuation/digits: keep Urdu letters and basic punctuation; remove others
        # We'll remove Latin letters and digits; keep Arabic-Urdu letters and basic punctuation
        self.allowed = re.compile(r"[^\u0600-\u06FF\s،؟۔\u0660-\u0669]")  # remove Latin/digits etc.
    def normalize(self, text: str) -> str:
        if not isinstance(text, str):
            return ""
        t = text.strip()
        t = self.diacritics_pattern.sub("", t)
        t = self.alef_pattern.sub("ا", t)
        t = self.yeh_pattern.sub("ی", t)
        t = self.kaf_pattern.sub("ک", t)
        # replace multiple spaces with single
        t = re.sub(r"\s+", " ", t).strip()
        # remove disallowed chars (Latin, punctuation except Urdu ones)
        t = self.allowed.sub("", t)
        # trim
        t = t.strip()
        return t

normalizer = UrduNormalizer()

# Preprocess dataset text strictly
def preprocess_df(df_in):
    q_norm = []
    a_norm = []
    for q,a in zip(df_in['question'], df_in['answer']):
        qn = normalizer.normalize(q)
        an = normalizer.normalize(a)
        # skip empty after normalization
        if qn == "" or an == "":
            continue
        q_norm.append(qn)
        a_norm.append(an)
    return pd.DataFrame({'question': q_norm, 'answer': a_norm})

df_clean = preprocess_df(df)
print("After normalization rows:", len(df_clean))


After normalization rows: 19998


In [21]:
# ----------------------------
# 5. Tokenization & Vocabulary (use whitespace tokens + special tokens)
# ----------------------------
SPECIAL_TOKENS = [config["PAD_TOKEN"], config["CLS_TOKEN"], config["SEP_TOKEN"], config["EOS_TOKEN"], config["UNK_TOKEN"]]

class Vocab:
    def __init__(self, min_freq=2):
        self.min_freq = min_freq
        self.token2idx = {}
        self.idx2token = {}
        # add specials
        for t in SPECIAL_TOKENS:
            self.add_token(t)
        self.freqs = Counter()
    def add_token(self, token):
        if token not in self.token2idx:
            idx = len(self.token2idx)
            self.token2idx[token] = idx
            self.idx2token[idx] = token
    def build(self, texts: List[str]):
        for s in texts:
            tokens = s.split()
            self.freqs.update(tokens)
        for tok, cnt in self.freqs.items():
            if cnt >= self.min_freq:
                self.add_token(tok)
    def encode(self, text: str, max_len:int):
        toks = text.split()[:max_len-2]  # leave room for CLS/EOS
        ids = [self.token2idx.get(t, self.token2idx[config["UNK_TOKEN"]]) for t in toks]
        return ids
    def decode(self, ids: List[int]):
        return " ".join([self.idx2token.get(i, config["UNK_TOKEN"]) for i in ids])
    def save(self, path):
        with open(path,'w',encoding='utf-8') as f:
            json.dump({'token2idx': self.token2idx, 'idx2token': self.idx2token}, f, ensure_ascii=False, indent=2)
    @classmethod
    def load(cls, path):
        with open(path,'r',encoding='utf-8') as f:
            d = json.load(f)
        v = cls()
        v.token2idx = d['token2idx']
        v.idx2token = {int(k):v for k,v in d['idx2token'].items()}
        return v

# Build vocab on training split only later; first split

In [22]:
# ----------------------------
# 6. Train/Val/Test split (80/10/10)
# ----------------------------
all_pairs = list(zip(df_clean['question'].tolist(), df_clean['answer'].tolist()))
random.seed(42)
random.shuffle(all_pairs)
N = len(all_pairs)
n_train = int(0.8 * N)
n_val = int(0.1 * N)
train_pairs = all_pairs[:n_train]
val_pairs = all_pairs[n_train:n_train+n_val]
test_pairs = all_pairs[n_train+n_val:]
print("Split sizes - train/val/test:", len(train_pairs), len(val_pairs), len(test_pairs))

# Build vocab from training texts only (both question + answer)
v = Vocab(min_freq=config["MIN_FREQ"])
train_texts = [q for q,a in train_pairs] + [a for q,a in train_pairs]
v.build(train_texts)
v.save(config["VOCAB_PATH"])
print("Vocab size:", len(v.token2idx))

PAD_ID = v.token2idx[config["PAD_TOKEN"]]
CLS_ID = v.token2idx[config["CLS_TOKEN"]]
SEP_ID = v.token2idx[config["SEP_TOKEN"]]
EOS_ID = v.token2idx[config["EOS_TOKEN"]]
UNK_ID = v.token2idx[config["UNK_TOKEN"]]


Split sizes - train/val/test: 15998 1999 2001
Vocab size: 10209


In [23]:
# ----------------------------
# 7. Dataset & Dataloader with special tokens
# ----------------------------
class UrduConvDataset(Dataset):
    def __init__(self, pairs, vocab: Vocab, max_len=50):
        self.pairs = pairs
        self.vocab = vocab
        self.max_len = max_len
    def __len__(self):
        return len(self.pairs)
    def __getitem__(self, idx):
        src, tgt = self.pairs[idx]
        # tokens
        src_ids = [CLS_ID] + v.encode(src, self.max_len) + [EOS_ID]
        tgt_ids = [CLS_ID] + v.encode(tgt, self.max_len) + [EOS_ID]
        # pad
        if len(src_ids) < self.max_len:
            src_ids = src_ids + [PAD_ID]*(self.max_len - len(src_ids))
        else:
            src_ids = src_ids[:self.max_len]
        if len(tgt_ids) < self.max_len:
            tgt_ids = tgt_ids + [PAD_ID]*(self.max_len - len(tgt_ids))
        else:
            tgt_ids = tgt_ids[:self.max_len]
        return torch.tensor(src_ids, dtype=torch.long), torch.tensor(tgt_ids, dtype=torch.long)

train_ds = UrduConvDataset(train_pairs, v, config["MAX_LEN"])
val_ds = UrduConvDataset(val_pairs, v, config["MAX_LEN"])
test_ds = UrduConvDataset(test_pairs, v, config["MAX_LEN"])
train_loader = DataLoader(train_ds, batch_size=config["BATCH_SIZE"], shuffle=True)
val_loader = DataLoader(val_ds, batch_size=config["BATCH_SIZE"], shuffle=False)
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False)



In [24]:
# ----------------------------
# 8. Transformer model from scratch
# ----------------------------
# Positional encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0,max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0,d_model,2).float()*(-math.log(10000.0)/d_model))
        pe[:,0::2] = torch.sin(position*div_term)
        pe[:,1::2] = torch.cos(position*div_term)
        self.register_buffer('pe', pe.unsqueeze(0))  # (1, max_len, d_model)
    def forward(self, x):
        # x: (batch, seq_len, d_model)
        seq_len = x.size(1)
        x = x + self.pe[:,:seq_len,:].to(x.device)
        return x

# Multi-head attention (from scratch)
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
    def forward(self, q, k, v, mask=None):
        # q,k,v: (batch, seq_len, d_model)
        bs = q.size(0)
        Q = self.w_q(q).view(bs, -1, self.num_heads, self.d_k).transpose(1,2)
        K = self.w_k(k).view(bs, -1, self.num_heads, self.d_k).transpose(1,2)
        V = self.w_v(v).view(bs, -1, self.num_heads, self.d_k).transpose(1,2)
        # scaled dot-product
        scores = torch.matmul(Q, K.transpose(-2,-1)) / math.sqrt(self.d_k)  # (bs, heads, qlen, klen)
        if mask is not None:
            # mask shape (bs, 1, 1, klen) or broadcastable
            scores = scores.masked_fill(mask==0, -1e9)
        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        out = torch.matmul(attn, V)  # (bs, heads, qlen, d_k)
        out = out.transpose(1,2).contiguous().view(bs, -1, self.d_model)  # (bs, qlen, d_model)
        out = self.w_o(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model)
        )
    def forward(self, x):
        return self.net(x)

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.ff = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.drop = nn.Dropout(dropout)
    def forward(self, x, mask=None):
        # x: (bs, seq, d_model)
        att = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.drop(att))
        ff = self.ff(x)
        x = self.norm2(x + self.drop(ff))
        return x

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.enc_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.ff = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.drop = nn.Dropout(dropout)
    def forward(self, x, enc_out, look_ahead_mask=None, padding_mask=None):
        # masked self
        att1 = self.self_attn(x, x, x, look_ahead_mask)
        x = self.norm1(x + self.drop(att1))
        # encoder-decoder
        att2 = self.enc_attn(x, enc_out, enc_out, padding_mask)
        x = self.norm2(x + self.drop(att2))
        ff = self.ff(x)
        x = self.norm3(x + self.drop(ff))
        return x

class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, d_ff, dropout, pad_idx):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_enc = PositionalEncoding(d_model, max_len=5000)
        self.layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout)
    def forward(self, src, mask=None):
        # src: (bs, seq)
        x = self.tok_emb(src) * math.sqrt(self.tok_emb.embedding_dim)
        x = self.pos_enc(x)
        x = self.dropout(x)
        for l in self.layers:
            x = l(x, mask)
        return x

class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, d_ff, dropout, pad_idx):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_enc = PositionalEncoding(d_model, max_len=5000)
        self.layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout)
        self.fc_out = nn.Linear(d_model, vocab_size)
    def forward(self, tgt, enc_out, look_ahead_mask=None, padding_mask=None):
        x = self.tok_emb(tgt) * math.sqrt(self.tok_emb.embedding_dim)
        x = self.pos_enc(x)
        x = self.dropout(x)
        for l in self.layers:
            x = l(x, enc_out, look_ahead_mask, padding_mask)
        out = self.fc_out(x)  # (bs, seq, vocab)
        return out

class TransformerModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_encoder_layers, num_decoder_layers, num_heads, d_ff, dropout, pad_idx):
        super().__init__()
        self.encoder = Encoder(src_vocab_size, d_model, num_encoder_layers, num_heads, d_ff, dropout, pad_idx)
        self.decoder = Decoder(tgt_vocab_size, d_model, num_decoder_layers, num_heads, d_ff, dropout, pad_idx)
        self.pad_idx = pad_idx
    def make_padding_mask(self, seq):
        # seq: (bs, seq_len)
        mask = (seq != self.pad_idx).unsqueeze(1).unsqueeze(2)  # (bs,1,1,seq_len)
        return mask
    def make_look_ahead_mask(self, size):
        mask = torch.triu(torch.ones((size,size), dtype=torch.bool), diagonal=1).to(next(self.parameters()).device)
        mask = ~mask  # True where allowed
        # convert to shape (1,1,size,size) when needed
        return mask
    def forward(self, src, tgt):
        # src,tgt: (bs, seq)
        src_mask = self.make_padding_mask(src)
        tgt_padding_mask = self.make_padding_mask(tgt)
        look = self.make_look_ahead_mask(tgt.size(1)).to(src.device)  # (size,size)
        # expand look to (bs,1,seq,seq) via broadcasting with tgt_padding_mask
        look = look.unsqueeze(0).unsqueeze(0) & tgt_padding_mask  # (bs,1,seq,seq)
        enc_out = self.encoder(src, src_mask)
        dec_out = self.decoder(tgt, enc_out, look, src_mask)
        return dec_out

# Create model
model = TransformerModel(
    src_vocab_size=len(v.token2idx),
    tgt_vocab_size=len(v.token2idx),
    d_model=config["D_MODEL"],
    num_encoder_layers=config["NUM_ENCODER_LAYERS"],
    num_decoder_layers=config["NUM_DECODER_LAYERS"],
    num_heads=config["NUM_HEADS"],
    d_ff=config["D_FF"],
    dropout=config["DROPOUT"],
    pad_idx=PAD_ID
).to(config["DEVICE"])

print("Model params:", sum(p.numel() for p in model.parameters()))


Model params: 30404065


In [25]:
# ----------------------------
# 9. Training Utilities
# ----------------------------
optimizer = torch.optim.Adam(model.parameters(), lr=config["LEARNING_RATE"], betas=(0.9,0.98), eps=1e-9)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_ID)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2)

def train_epoch(model, loader, optimizer, criterion, device, clip=1.0, teacher_forcing_ratio=0.5):
    model.train()
    total_loss=0
    pbar = tqdm(loader, desc="Train", leave=False)
    for src, tgt in pbar:
        src, tgt = src.to(device), tgt.to(device)
        # input to decoder: all tokens except last
        tgt_input = tgt[:, :-1]
        tgt_output = tgt[:, 1:]
        optimizer.zero_grad()
        preds = model(src, tgt_input)  # (bs, seq, vocab)
        preds_reshaped = preds.reshape(-1, preds.size(-1))
        loss = criterion(preds_reshaped, tgt_output.reshape(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        total_loss += loss.item()
        pbar.set_postfix(loss=f"{loss.item():.4f}")
    return total_loss / len(loader)


In [26]:
# ----------------------------
# 10. Evaluation metrics functions
# ----------------------------
def compute_metrics(model, loader, vocab_obj, device, max_examples=None):
    model.eval()
    refs = []
    hyps = []
    total_loss = 0.0
    with torch.no_grad():
        for i, (src, tgt) in enumerate(tqdm(loader, desc="Eval", leave=False)):
            src, tgt = src.to(device), tgt.to(device)
            # generate using beam search for quality
            hyp = generate_sentence(model, src[0:1], vocab_obj, device, method='beam', beam_size=3, max_len=config["MAX_LEN"])
            # reference
            ref = vocab_obj.decode([int(x) for x in tgt[0].cpu().tolist()]).replace(config["CLS_TOKEN"],"").replace(config["EOS_TOKEN"],"").strip()
            refs.append(ref)
            hyps.append(hyp)
            # compute loss for perplexity
            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]
            preds = model(src, tgt_input)
            loss = criterion(preds.reshape(-1, preds.size(-1)), tgt_output.reshape(-1))
            total_loss += loss.item()
            if max_examples and i+1>=max_examples:
                break
    # BLEU (sacreBLEU expects lists)
    bleu = sacrebleu.corpus_bleu(hyps, [refs]).score
    # chrF
    chrf = sacrebleu.corpus_chrf(hyps, [refs]).score
    # ROUGE-L
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_l = np.mean([scorer.score(r,h)['rougeL'].fmeasure for r,h in zip(refs,hyps)])*100
    avg_loss = total_loss / len(loader)
    perplexity = math.exp(avg_loss) if avg_loss < 100 else float('inf')
    return {'bleu':bleu, 'chrf':chrf, 'rouge_l':rouge_l, 'perplexity':perplexity, 'examples': list(zip(refs,hyps))[:5]}



In [27]:
# ----------------------------
# 11. Generation (greedy + beam)
# ----------------------------
def generate_sentence(model, src_tensor, vocab_obj, device, method='greedy', beam_size=3, max_len=50):
    # src_tensor: (1, seq)
    model.eval()
    with torch.no_grad():
        enc_out = model.encoder(src_tensor, model.make_padding_mask(src_tensor))
        if method == 'greedy':
            # start with CLS
            cur = torch.tensor([[CLS_ID]], device=device)
            for _ in range(max_len-1):
                out = model.decoder(cur, enc_out, model.make_look_ahead_mask(cur.size(1)).unsqueeze(0).unsqueeze(0).to(device), model.make_padding_mask(src_tensor))
                logits = out[:, -1, :]  # (1, vocab)
                next_token = logits.argmax(-1).item()
                cur = torch.cat([cur, torch.tensor([[next_token]], device=device)], dim=1)
                if next_token == EOS_ID:
                    break
            res_ids = cur.squeeze(0).cpu().tolist()
            return vocab_obj.decode([i for i in res_ids if i not in (PAD_ID,)])
        elif method == 'beam':
            # beam search
            beams = [([CLS_ID], 0.0)]
            for _ in range(max_len-1):
                new_beams = []
                for seq, score in beams:
                    if seq[-1] == EOS_ID:
                        new_beams.append((seq, score))
                        continue
                    cur_tensor = torch.tensor([seq], device=device)
                    out = model.decoder(cur_tensor, enc_out, model.make_look_ahead_mask(cur_tensor.size(1)).unsqueeze(0).unsqueeze(0).to(device), model.make_padding_mask(src_tensor))
                    logits = out[:, -1, :]
                    log_probs = F.log_softmax(logits, dim=-1).squeeze(0)
                    topk = torch.topk(log_probs, beam_size)
                    for k in range(beam_size):
                        tok = int(topk.indices[k].item())
                        sc = score + float(topk.values[k].item())
                        new_beams.append((seq+[tok], sc))
                new_beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]
                beams = new_beams
            best_seq = beams[0][0]
            return vocab_obj.decode([i for i in best_seq if i not in (PAD_ID,)])
        else:
            raise ValueError("Invalid method")

In [28]:
# ----------------------------
# 12. Training loop with validation & save best by BLEU
# ----------------------------
best_val_metric = -1e9
for epoch in range(config["NUM_EPOCHS"]):
    start = time.time()
    train_loss = train_epoch(model, train_loader, optimizer, criterion, config["DEVICE"], clip=1.0, teacher_forcing_ratio=0.5)
    val_metrics = compute_metrics(model, val_loader, v, config["DEVICE"], max_examples=200)  # sample 200 for speed each epoch
    val_bleu = val_metrics['bleu']
    scheduler.step(val_bleu)
    epoch_time = time.time() - start
    print(f"Epoch {epoch+1}/{config['NUM_EPOCHS']} | Train Loss: {train_loss:.4f} | Val BLEU: {val_bleu:.2f} | Time: {epoch_time:.1f}s")
    print("Val metrics:", val_metrics)
    # Save best
    if val_bleu > best_val_metric:
        best_val_metric = val_bleu
        torch.save(model.state_dict(), config["MODEL_SAVE_PATH"])
        print("Saved best model with BLEU:", best_val_metric)

Epoch 1/20 | Train Loss: 5.4813 | Val BLEU: 0.00 | Time: 39.9s
Val metrics: {'bleu': 0.0001127848139930023, 'chrf': 2.8642006478528, 'rouge_l': np.float64(0.0), 'perplexity': 82.26550789500415, 'examples': [('اردوغان پر کرپشن کے الزامات ییں۔  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] دوسروں کے طور پر ییں۔ [EOS]'), ('[UNK] ولیمز امیر ترین خاتون ایتھلیٹ قرار  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] انیوں پی ایس [EOS]'), ('کیا انییں سال بھر زنجیروں میں نییں جکڑا جا سکتا؟  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

Epoch 2/20 | Train Loss: 3.8263 | Val BLEU: 0.00 | Time: 40.1s
Val metrics: {'bleu': 0.0005635888942054769, 'chrf': 4.643566535127758, 'rouge_l': np.float64(0.0), 'perplexity': 31.374605030213772, 'examples': [('اردوغان پر کرپشن کے الزامات ییں۔  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] ییں۔ [EOS]'), ('[UNK] ولیمز امیر ترین خاتون ایتھلیٹ قرار  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] پاکستان اسٹاک ایکسچینج [EOS]'), ('کیا انییں سال بھر زنجیروں میں نییں جکڑا جا سکتا؟  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

Epoch 3/20 | Train Loss: 2.8469 | Val BLEU: 0.00 | Time: 40.5s
Val metrics: {'bleu': 0.0013230957451474696, 'chrf': 5.722591795360107, 'rouge_l': np.float64(0.0), 'perplexity': 15.452786035332089, 'examples': [('اردوغان پر کرپشن کے الزامات ییں۔  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] جنوبی افریقی کے ییں۔ [EOS]'), ('[UNK] ولیمز امیر ترین خاتون ایتھلیٹ قرار  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] اسٹریلین سنٹرل ڈے لایٹ ٹایم [EOS]'), ('کیا انییں سال بھر زنجیروں میں نییں جکڑا جا سکتا؟  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

Epoch 4/20 | Train Loss: 2.1295 | Val BLEU: 0.00 | Time: 39.1s
Val metrics: {'bleu': 0.0009264268631105228, 'chrf': 6.427477808943626, 'rouge_l': np.float64(0.0), 'perplexity': 10.40175738513695, 'examples': [('اردوغان پر کرپشن کے الزامات ییں۔  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] کرپشن کے الزامات پر کرپشن ییں۔ [EOS]'), ('[UNK] ولیمز امیر ترین خاتون ایتھلیٹ قرار  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] امیر محترم [EOS]'), ('کیا انییں سال بھر زنجیروں میں نییں جکڑا جا سکتا؟  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

Epoch 5/20 | Train Loss: 1.4751 | Val BLEU: 0.00 | Time: 40.8s
Val metrics: {'bleu': 0.003641917467436672, 'chrf': 8.05047548564375, 'rouge_l': np.float64(0.0), 'perplexity': 6.754950955780499, 'examples': [('اردوغان پر کرپشن کے الزامات ییں۔  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] کرپشن کے الزامات پر کرپشن ییں۔ [EOS]'), ('[UNK] ولیمز امیر ترین خاتون ایتھلیٹ قرار  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] امیر محترم جنرل ضیاء الحق [EOS]'), ('کیا انییں سال بھر زنجیروں میں نییں جکڑا جا سکتا؟  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

...

Epoch 16/20 | Train Loss: 0.3597 | Val BLEU: 0.01 | Time: 40.5s
Val metrics: {'bleu': 0.01385016010386425, 'chrf': 11.536485229446935, 'rouge_l': np.float64(0.0), 'perplexity': 3.543643286926181, 'examples': [('اردوغان پر کرپشن کے الزامات ییں۔  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] کرپشن کے الزامات پر کرپشن پر کرپشن ییں۔ [EOS]'), ('[UNK] ولیمز امیر ترین خاتون ایتھلیٹ قرار  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] امیر محترم جنرل حمید گل بنیں قرار [EOS]'), ('کیا انییں سال بھر زنجیروں میں نییں جکڑا جا سکتا؟  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

Epoch 17/20 | Train Loss: 0.3434 | Val BLEU: 0.01 | Time: 40.6s
Val metrics: {'bleu': 0.010956226042856872, 'chrf': 11.501774949853822, 'rouge_l': np.float64(0.0), 'perplexity': 3.5348079734980247, 'examples': [('اردوغان پر کرپشن کے الزامات ییں۔  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] کرپشن کے الزامات پر کرپشن پر کرپشن ییں۔ [EOS]'), ('[UNK] ولیمز امیر ترین خاتون ایتھلیٹ قرار  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] امیر محترم جنرل حمید گل بنیں قرار [EOS]'), ('کیا انییں سال بھر زنجیروں میں نییں جکڑا جا سکتا؟  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

Epoch 18/20 | Train Loss: 0.3350 | Val BLEU: 0.01 | Time: 40.3s
Val metrics: {'bleu': 0.012432167723622881, 'chrf': 11.625172701737084, 'rouge_l': np.float64(0.0), 'perplexity': 3.520583803285818, 'examples': [('اردوغان پر کرپشن کے الزامات ییں۔  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] کرپشن کے الزامات پر کرپشن پر کرپشن ییں۔ [EOS]'), ('[UNK] ولیمز امیر ترین خاتون ایتھلیٹ قرار  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] امیر محترم جنرل حمید گل بنیں قرار [EOS]'), ('کیا انییں سال بھر زنجیروں میں نییں جکڑا جا سکتا؟  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

Epoch 19/20 | Train Loss: 0.3274 | Val BLEU: 0.01 | Time: 40.4s
Val metrics: {'bleu': 0.012265101579231736, 'chrf': 11.63703608887304, 'rouge_l': np.float64(0.0), 'perplexity': 3.5038652657070726, 'examples': [('اردوغان پر کرپشن کے الزامات ییں۔  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] کرپشن کے الزامات پر کرپشن پر کرپشن ییں۔ [EOS]'), ('[UNK] ولیمز امیر ترین خاتون ایتھلیٹ قرار  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] امیر محترم جنرل حمید گل بنیں قرار [EOS]'), ('کیا انییں سال بھر زنجیروں میں نییں جکڑا جا سکتا؟  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

Epoch 20/20 | Train Loss: 0.3187 | Val BLEU: 0.01 | Time: 40.6s
Val metrics: {'bleu': 0.012147596475003831, 'chrf': 11.770049553768608, 'rouge_l': np.float64(0.0), 'perplexity': 3.5101920593219167, 'examples': [('اردوغان پر کرپشن کے الزامات ییں۔  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] کرپشن کے الزامات پر کرپشن پر کرپشن ییں۔ [EOS]'), ('[UNK] ولیمز امیر ترین خاتون ایتھلیٹ قرار  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] امیر محترم جنرل حمید گل بنیں قرار [EOS]'), ('کیا انییں سال بھر زنجیروں میں نییں جکڑا جا سکتا؟  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [29]:
# ----------------------------
# 13. Final evaluation on test set
# ----------------------------
print("Loading best model and evaluating on test set...")
model.load_state_dict(torch.load(config["MODEL_SAVE_PATH"], map_location=config["DEVICE"]))
test_metrics = compute_metrics(model, test_loader, v, config["DEVICE"], max_examples=None)
print("\n--- TEST METRICS ---")
print(f"BLEU: {test_metrics['bleu']:.2f}")
print(f"ROUGE-L: {test_metrics['rouge_l']:.2f}")
print(f"chrF: {test_metrics['chrf']:.2f}")
print(f"Perplexity: {test_metrics['perplexity']:.2f}")
print("Sample examples (ref, hyp):")
for r,h in test_metrics['examples']:
    print("REF:", r)
    print("HYP:", h)
    print("-----")

Loading best model and evaluating on test set...



--- TEST METRICS ---
BLEU: 0.01
ROUGE-L: 0.00
chrF: 10.61
Perplexity: 3.84
Sample examples (ref, hyp):
REF: کسی تیسرے فریق پی۔  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
HYP: [CLS] کسی خاص مالے [EOS]
-----
REF: میری طرح کا [UNK] اس [UNK] [UNK] میں [UNK] جاتا یے۔  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
HYP: [CLS] اس طرح امن میں مضامین لکھنے کا یے؟ [EOS]
-----
REF: ماییکروسافٹ کے مشیور زمانی اپریٹنگ سسٹم ونڈوز کا نیا ورژن اب کمپنی ویب ساییٹ پر پری ارڈر کے لیے دستیاب یے  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [49]:
# ----------------------------
# 14. Streamlit + ngrok interface (Colab friendly)
# ----------------------------
# The Streamlit app uses the saved best model. Uses ngrok to expose public URL.
# In Colab run this cell in its own cell to launch app. For local/Kaggle adjust accordingly.

streamlit_script = r'''
import streamlit as st, torch, math, re, json
from pathlib import Path
# minimal app imports
st.set_page_config(page_title="Urdu Transformer Chatbot", layout="centered")
st.title("Urdu Chatbot (Transformer)")
st.markdown("Type Urdu text and get a response. Uses saved Transformer model.")

# load vocab - expect files in working dir
try:
    with open("vocab_tokens.json","r",encoding="utf-8") as f:
        d=json.load(f)
    token2idx = d['token2idx']
    idx2token = {int(k):v for k,v in d['idx2token'].items()}
    PAD = token2idx["[PAD]"]
    CLS = token2idx["[CLS]"]
    EOS = token2idx["[EOS]"]
    UNK = token2idx["[UNK]"]
except FileNotFoundError:
    st.error("vocab_tokens.json not found. Make sure to run the training steps first.")
    st.stop()


# re-create small normalizer (same as training)
class Normalizer:
    def __init__(self):
        self.diacritics_pattern = re.compile(r"[\u064B-\u0652\u0670\u0640]")
        self.alef_pattern = re.compile(r"[\u0622\u0623\u0625\u0671\u0672\u0673]")
        self.yeh_pattern = re.compile(r"[\u0626\u06CC\u06C0\u06C1\u06C2\u064A]")
        self.kaf_pattern = re.compile(r"[\u0643\u06AA]")
        self.allowed = re.compile(r"[^\u0600-\u06FF\s،؟۔\u0660-\u0669]")
    def normalize(self,t):
        if not isinstance(t, str):
            return ""
        t = t.strip()
        t = self.diacritics_pattern.sub("", t)
        t = self.alef_pattern.sub("ا", t)
        t = self.yeh_pattern.sub("ی", t)
        t = self.kaf_pattern.sub("ک", t)
        t = re.sub(r"\s+"," ", t).strip()
        t = self.allowed.sub("", t)
        return t.strip()
norm = Normalizer()

# small helper encode/decode
def encode_text(s, max_len=50):
    toks = s.split()[:max_len-2]
    ids = [CLS] + [token2idx.get(t, token2idx['[UNK]']) for t in toks] + [EOS]
    if len(ids) < max_len:
        ids = ids + [token2idx['[PAD]']]*(max_len - len(ids))
    else:
        ids = ids[:max_len]
    return torch.tensor([ids], dtype=torch.long)

# Modified decode_ids to accept 'self'
def decode_ids(self, ids):
    return " ".join([idx2token.get(int(i), "[UNK]") for i in ids if int(i) not in (token2idx['[PAD]'],)])


# --- Model Architecture (Copied from notebook) ---
# Positional encoding
class PositionalEncoding(torch.nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0,max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0,d_model,2).float()*(-math.log(10000.0)/d_model))
        pe[:,0::2] = torch.sin(position*div_term)
        pe[:,1::2] = torch.cos(position*div_term)
        self.register_buffer('pe', pe.unsqueeze(0))  # (1, max_len, d_model)
    def forward(self, x):
        # x: (batch, seq_len, d_model)
        seq_len = x.size(1)
        x = x + self.pe[:,:seq_len,:].to(x.device)
        return x

# Multi-head attention (from scratch)
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        self.w_q = torch.nn.Linear(d_model, d_model)
        self.w_k = torch.nn.Linear(d_model, d_model)
        self.w_v = torch.nn.Linear(d_model, d_model)
        self.w_o = torch.nn.Linear(d_model, d_model)
        self.dropout = torch.nn.Dropout(dropout)
    def forward(self, q, k, v, mask=None):
        # q,k,v: (batch, seq_len, d_model)
        bs = q.size(0)
        Q = self.w_q(q).view(bs, -1, self.num_heads, self.d_k).transpose(1,2)
        K = self.w_k(k).view(bs, -1, self.num_heads, self.d_k).transpose(1,2)
        V = self.w_v(v).view(bs, -1, self.num_heads, self.d_k).transpose(1,2)
        # scaled dot-product
        scores = torch.matmul(Q, K.transpose(-2,-1)) / math.sqrt(self.d_k)  # (bs, heads, qlen, klen)
        if mask is not None:
            # mask shape (bs, 1, 1, klen) or broadcastable
            scores = scores.masked_fill(mask==0, -1e9)
        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        out = torch.matmul(attn, V)  # (bs, heads, qlen, d_k)
        out = out.transpose(1,2).contiguous().view(bs, -1, self.d_model)  # (bs, qlen, d_model)
        out = self.w_o(out)
        return out

class FeedForward(torch.nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(d_model, d_ff),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(d_ff, d_model)
        )
    def forward(self, x):
        return self.net(x)

class EncoderLayer(torch.nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.ff = FeedForward(d_model, d_ff, dropout)
        self.norm1 = torch.nn.LayerNorm(d_model)
        self.norm2 = torch.nn.LayerNorm(d_model)
        self.drop = torch.nn.Dropout(dropout)
    def forward(self, x, mask=None):
        # x: (bs, seq, d_model)
        att = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.drop(att))
        ff = self.ff(x)
        x = self.norm2(x + self.drop(ff))
        return x

class DecoderLayer(torch.nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.enc_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.ff = FeedForward(d_model, d_ff, dropout)
        self.norm1 = torch.nn.LayerNorm(d_model)
        self.norm2 = torch.nn.LayerNorm(d_model)
        self.norm3 = torch.nn.LayerNorm(d_model)
        self.drop = torch.nn.Dropout(dropout)
    def forward(self, x, enc_out, look_ahead_mask=None, padding_mask=None):
        # masked self
        att1 = self.self_attn(x, x, x, look_ahead_mask)
        x = self.norm1(x + self.drop(att1))
        # encoder-decoder
        att2 = self.enc_attn(x, enc_out, enc_out, padding_mask)
        x = self.norm2(x + self.drop(att2))
        ff = self.ff(x)
        x = self.norm3(x + self.drop(ff))
        return x

class Encoder(torch.nn.Module):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, d_ff, dropout, pad_idx):
        super().__init__()
        self.tok_emb = torch.nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_enc = PositionalEncoding(d_model, max_len=5000)
        self.layers = torch.nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.dropout = torch.nn.Dropout(dropout)
    def forward(self, src, mask=None):
        # src: (bs, seq)
        x = self.tok_emb(src) * math.sqrt(self.tok_emb.embedding_dim)
        x = self.pos_enc(x)
        x = self.dropout(x)
        for l in self.layers:
            x = l(x, mask)
        return x

class Decoder(torch.nn.Module):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, d_ff, dropout, pad_idx):
        super().__init__()
        self.tok_emb = torch.nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_enc = PositionalEncoding(d_model, max_len=5000)
        self.layers = torch.nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.dropout = torch.nn.Dropout(dropout)
        self.fc_out = torch.nn.Linear(d_model, vocab_size)
    def forward(self, tgt, enc_out, look_ahead_mask=None, padding_mask=None):
        x = self.tok_emb(tgt) * math.sqrt(self.tok_emb.embedding_dim)
        x = self.pos_enc(x)
        x = self.dropout(x)
        for l in self.layers:
            x = l(x, enc_out, look_ahead_mask, padding_mask)
        out = self.fc_out(x)  # (bs, seq, vocab)
        return out

class TransformerModel(torch.nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_encoder_layers, num_decoder_layers, num_heads, d_ff, dropout, pad_idx):
        super().__init__()
        self.encoder = Encoder(src_vocab_size, d_model, num_encoder_layers, num_heads, d_ff, dropout, pad_idx)
        self.decoder = Decoder(tgt_vocab_size, d_model, num_decoder_layers, num_heads, d_ff, dropout, pad_idx)
        self.pad_idx = pad_idx
    def make_padding_mask(self, seq):
        # seq: (bs, seq_len)
        mask = (seq != self.pad_idx).unsqueeze(1).unsqueeze(2)  # (bs,1,1,seq_len)
        return mask
    def make_look_ahead_mask(self, size):
        mask = torch.triu(torch.ones((size,size), dtype=torch.bool), diagonal=1).to(next(self.parameters()).device)
        mask = ~mask  # True where allowed
        # convert to shape (1,1,size,size) when needed
        return mask
    def forward(self, src, tgt):
        # src,tgt: (bs, seq)
        src_mask = self.make_padding_mask(src)
        tgt_padding_mask = self.make_padding_mask(tgt)
        look = self.make_look_ahead_mask(tgt.size(1)).to(src.device)  # (size,size)
        # expand look to (bs,1,seq,seq) via broadcasting with tgt_padding_mask
        look = look.unsqueeze(0).unsqueeze(0) & tgt_padding_mask  # (bs,1,seq,seq)
        enc_out = self.encoder(src, src_mask)
        dec_out = self.decoder(tgt, enc_out, look, src_mask)
        return dec_out

# --- Generation Function (Copied from notebook) ---
def generate_sentence(model, src_tensor, vocab_obj, device, method='greedy', beam_size=3, max_len=50):
    # src_tensor: (1, seq)
    model.eval()
    with torch.no_grad():
        enc_out = model.encoder(src_tensor, model.make_padding_mask(src_tensor))
        if method == 'greedy':
            # start with CLS
            cur = torch.tensor([[CLS]], device=device)
            for _ in range(max_len-1):
                out = model.decoder(cur, enc_out, model.make_look_ahead_mask(cur.size(1)).unsqueeze(0).unsqueeze(0).to(device), model.make_padding_mask(src_tensor))
                logits = out[:, -1, :]  # (1, vocab)
                next_token = logits.argmax(-1).item()
                cur = torch.cat([cur, torch.tensor([[next_token]], device=device)], dim=1)
                if next_token == EOS:
                    break
            res_ids = cur.squeeze(0).cpu().tolist()
            # Call decode_ids as a method of vocab_wrapper
            return vocab_obj.decode(res_ids)
        elif method == 'beam':
            # beam search
            beams = [([CLS], 0.0)]
            for _ in range(max_len-1):
                new_beams = []
                for seq, score in beams:
                    if seq[-1] == EOS:
                        new_beams.append((seq, score))
                        continue
                    cur_tensor = torch.tensor([seq], device=device)
                    out = model.decoder(cur_tensor, enc_out, model.make_look_ahead_mask(cur_tensor.size(1)).unsqueeze(0).unsqueeze(0).to(device), model.make_padding_mask(src_tensor))
                    logits = out[:, -1, :]
                    log_probs = torch.nn.functional.log_softmax(logits, dim=-1).squeeze(0)
                    topk = torch.topk(log_probs, beam_size)
                    for k in range(beam_size):
                        tok = int(topk.indices[k].item())
                        sc = score + float(topk.values[k].item())
                        new_beams.append((seq+[tok], sc))
                new_beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]
                beams = new_beams
            best_seq = beams[0][0]
            # Call decode_ids as a method of vocab_wrapper
            return vocab_obj.decode(best_seq)
        else:
            raise ValueError("Invalid method")


# --- Model Loading ---
best_model_path = "best_transformer_urdu.pt" # Ensure this matches the save path
try:
    # Need to match the model architecture used during training
    # Refer to the config used when saving the model
    # Assuming config was D_MODEL=512, NUM_HEADS=2, NUM_ENCODER_LAYERS=2, NUM_DECODER_LAYERS=2, D_FF=2048, DROPOUT=0.1
    model_params = {
        "src_vocab_size": len(token2idx),
        "tgt_vocab_size": len(token2idx),
        "d_model": 512,
        "num_encoder_layers": 2,
        "num_decoder_layers": 2,
        "num_heads": 2,
        "d_ff": 2048,
        "dropout": 0.1,
        "pad_idx": PAD
    }
    net = TransformerModel(**model_params).to(torch.device('cpu')) # Load to CPU for Streamlit
    net.load_state_dict(torch.load(best_model_path, map_location=torch.device('cpu')))
    net.eval()
except FileNotFoundError:
    st.error(f"{best_model_path} not found. Make sure to train and save the model first.")
    st.stop()
except Exception as e:
    st.error("Could not load model state dict: " + str(e))
    st.stop()


# conversation state
if 'history' not in st.session_state:
    st.session_state.history = []

col1, col2 = st.columns([3,1])
method = col2.selectbox("Decoding", ["beam","greedy"])
inp = st.text_area("سوال لکھیں (Urdu):", height=120)
if st.button("Send"):
    if not inp.strip():
        st.warning("Enter text.")
    else:
        st.session_state.history.append(("You", inp))
        # removed the net check as it should be loaded or app stops
        s = norm.normalize(inp)
        src = encode_text(s, max_len=50)
        # Pass a simple object with decode method instead of full Vocab object
        vocab_wrapper = type('V',(),{'decode': decode_ids})()
        resp = generate_sentence(net, src, vocab_wrapper, torch.device('cpu'), method=method, beam_size=3, max_len=50)
        st.session_state.history.append(("Bot", resp))

for who, msg in st.session_state.history[-10:]:
    if who=="You":
        st.markdown(f"**You:** {msg}")
    else:
        st.markdown(f"**Bot:** {msg}")

'''

In [50]:
# Save streamlit script to file
with open("streamlit_app.py","w",encoding="utf-8") as f:
    f.write(streamlit_script)

print("Streamlit app written to streamlit_app.py")
print("To run Streamlit + ngrok in Colab, run the following commands in a cell (separately):\n")
print("!pip install pyngrok")
print("from pyngrok import ngrok")
print("!streamlit run streamlit_app.py &")
print("ngrok_tunnel = ngrok.connect(8501); print('Public URL:', ngrok_tunnel.public_url)")


Streamlit app written to streamlit_app.py
To run Streamlit + ngrok in Colab, run the following commands in a cell (separately):

!pip install pyngrok
from pyngrok import ngrok
!streamlit run streamlit_app.py &
ngrok_tunnel = ngrok.connect(8501); print('Public URL:', ngrok_tunnel.public_url)


In [51]:
# ----------------------------
# End of script
# ----------------------------
print("Script complete. Model saved to:", config["MODEL_SAVE_PATH"])
print("Vocab saved to:", config["VOCAB_PATH"])

Script complete. Model saved to: best_transformer_urdu.pt
Vocab saved to: vocab_tokens.json


In [52]:
!pip install pyngrok streamlit
from pyngrok import ngrok
from google.colab import userdata

# Get the ngrok authtoken from Colab secrets
NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTH_TOKEN")
if NGROK_AUTH_TOKEN is None:
    raise ValueError("NGROK_AUTH_TOKEN not found in Colab secrets. Please add it as instructed above.")

# Set the ngrok authtoken
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# start streamlit in background
get_ipython().system_raw('streamlit run streamlit_app.py --server.port 8501 &')
# make public
ngrok_tunnel = ngrok.connect(8501)
print("Public URL:", ngrok_tunnel.public_url)

Public URL: https://postauditory-norah-backwards.ngrok-free.dev
